In [1]:
import pandas as pd
import numpy as np
import math
import os

def printCategory(category_dict):
    for category in category_dict:
        print(category)
        if category_dict[category] is not None:
            sub_dic = category_dict[category]
            printCategory(sub_dic)
            
def createLevelColumns(df, levels, new_columns):
    for i in range(len(df)):
        
        df_level1, df_level2 = "NULL","NULL"
        
        df_level1, nextLevels = findNotNullLevel(df, i, levels)
        if nextLevels is not None:
            df_level2, nextLevels = findNotNullLevel(df, i, nextLevels)
        
        most_specific_category = "NULL"
        
        if df_level2 != "NULL":
            df_level2 = df_level2 + "__" + df_level1
        
        if df_level2 != "NULL":
            most_specific_category = df_level2
        elif df_level1 != "NULL":
            most_specific_category = df_level1
        
        df.iloc[i, df.columns.get_loc('level1')] = df_level1
        df.iloc[i, df.columns.get_loc('level2')] = df_level2
        df.iloc[i, df.columns.get_loc('mostSpecificCategory')] = most_specific_category
        
def findNotNullLevel(df, i, levels):
    for level in levels:
        if df.iloc[i][level] == "NULL":
            continue
        else:
            return df.iloc[i][level], levels[level]
    return "NULL", None

def unionTwoLists(list1, list2):
    for category in list1:
        if category not in list2:
            list2.append(category)
    return list2

def checkNULL(checked_list):
    for item in checked_list:
        if item == "NULL":
            print("Contains NULL")
            return
    print("Not contains NULL")

def create_name_id_dictionary(name_list):
    count = 1
    dic_name_id = {}
    for name in name_list:
        if name not in dic_name_id and name != "NULL":
            dic_name_id[name] = count
            count = count + 1
    return dic_name_id

In [3]:
path = os.path.abspath(os.getcwd())
df = pd.read_csv(path + "/Data2/Last Names.csv")
df = df.dropna(axis = 0, how = 'all')
df = df.where(df.notnull(), "NULL")
print(df.columns)
df.head()

Index(['村志代码 Gazetteer Code', '村志书名 Gazetteer Title',
       '姓氏总数 Total Number of Last Names in Village', 'name1', 'name2', 'name3',
       'name4', 'name5', 'pinyin1', 'pinyin2', 'pinyin3', 'pinyin4',
       'pinyin5'],
      dtype='object')


,村志代码 Gazetteer Code,村志书名 Gazetteer Title,姓氏总数 Total Number of Last Names in Village,name1,name2,name3,name4,name5,pinyin1,pinyin2,pinyin3,pinyin4,pinyin5
0,1,太平店村志,17,马,范,李,梁,姚,Ma,Fan,Li,Liang,Yao
1,2,叶店村志,45,叶,邓,杨,陈,梅,Ye,Deng,Yang,Chen,Mei
2,3,马鞭田村志,59,张,陈,杨,杜,王,Zhang,Chen,Yang,Du,Wang
3,4,万秀村志,117,邓,陈,黄,梁,林,Deng,Chen,Huang,Liang,Lin
4,5,光淀村志,26,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [5]:
# {'马': {'id': 1, 'pinyin':'Ma'}, '李': ...}
name_pingying_id = {}
count = 1
name_heading = list(df.columns)[3:8]
pinyin_heading = list(df.columns)[8:13]
for i in range(len(df)):
    for j in range(len(name_heading)):
        name = df[name_heading[j]][i]
        pinyin = df[pinyin_heading[j]][i]
        if name not in name_pingying_id:
            name_pingying_id[name]['id'] = count
            name_pingying_id[name]['pinyin'] = pinyin
            count = count + 1

df_last_name = pd.DataFrame(columns = ['gazetteerId', 'totalNumber', 'firstNameId', 'secondNameId', 'thirdNameId', 'fourthNameId','fifthNameId'])
dic_last_name = {'gazetteerId':[], 'totalNumber':[] 'firstNameId':[], 'secondNameId':[], 'thirdNameId':[], 'fourthNameId':[], 'fifthNameId':[]}





['name1', 'name2', 'name3', 'name4', 'name5']

In [56]:
def create_name_id_dictionary(name_list):
    count = 1
    dic_name_id = {}
    for name in name_list:
        if name not in dic_name_id and name != "NULL":
            dic_name_id[name] = count
            count = count + 1
    return dic_name_id

# create dict "dic_category_id" store { category_name : id}
total_categories = ['海拔 (米) Altitude', '平均降水量 Average Yearly Precipitation Amount', '平均温度 Average Yearly Temperature']
dic_category_id = create_name_id_dictionary(total_categories)

# create dict "dic_unit_id" store { unit_name : id}
unit_list = [cat for cat in df_total['unit'].astype('category').unique()]
dic_unit_id = create_name_id_dictionary(unit_list)

print(dic_category_id)
print(dic_unit_id)

{'海拔 (米) Altitude': 1, '平均降水量 Average Yearly Precipitation Amount': 2, '平均温度 Average Yearly Temperature': 3}
{'米 meter': 1, '毫米 millimeter': 2, '没有相关数据 No Available Data': 3, '°C': 4}


In [57]:
for category in dic_category_id:
    df_total = df_total.replace(category, dic_category_id[category])
    
for unit in dic_unit_id:
    df_total = df_total.replace(unit, dic_unit_id[unit])

# create natural_environment_df
natural_environment_df = df_total.copy()
natural_environment_df.columns = columns = ['gazetteerId', 'category', 'data', 'unitId']
natural_environment_df.head()

,gazetteerId,category,data,unitId
0,1,1,NULL,1
1,3,1,1225,1
2,4,1,NULL,1
3,5,1,NULL,1
4,6,1,61,1


In [58]:
# create economyCategory_df
category_df = pd.DataFrame(columns = ['id', 'name', 'parentId'])
dic_for_category_df = {'id':[], 'name':[], 'parentId':[]}

for category in dic_category_id:
    child_parent = category.split('__', 1)
    name = child_parent[0]
    if len(child_parent) == 1:
        dic_for_category_df['id'].append(dic_category_id[category])
        dic_for_category_df['name'].append(name)
        dic_for_category_df['parentId'].append("NULL")
    else:
        parentId = dic_category_id[child_parent[1]]
        dic_for_category_df['id'].append(dic_category_id[category])
        dic_for_category_df['name'].append(name)
        dic_for_category_df['parentId'].append(parentId)
        
for attribute in category_df.columns:
    category_df[attribute] = dic_for_category_df[attribute]

category_df.head()

,id,name,parentId
0,1,海拔 (米) Altitude,NULL
1,2,平均降水量 Average Yearly Precipitation Amount,NULL
2,3,平均温度 Average Yearly Temperature,NULL


In [59]:
dic_unit_df = {'id':[], 'name':[]}

for unit in dic_unit_id:
    dic_unit_df['name'].append(unit)
    dic_unit_df['id'].append(dic_unit_id[unit])
    
unit_id_df = pd.DataFrame(dic_unit_df,columns = ['id', 'name'])
unit_id_df.head()

,id,name
0,1,米 meter
1,2,毫米 millimeter
2,3,没有相关数据 No Available Data
3,4,°C


In [60]:
unit_id_df.to_csv('natural_env_unit_df.csv',index = False, na_rep = "NULL")

In [61]:
natural_environment_df.to_csv('natural_env_df.csv', index = False, na_rep = "NULL")

In [62]:
category_df.to_csv('natural_env_category_df.csv', index = False, na_rep = "NULL")